In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from tqdm import tqdm

np.random.seed(1024)
VERBOSE = False

## 1. IO


In [ ]:
network = pd.read_csv("../reproduction/data/0_topology.csv")
task = pd.read_csv("../reproduction/data/0_medium_task.csv")

In [ ]:
applicaiton_1 = task[task['start'] == 8]

In [ ]:
gcd = np.gcd.reduce(applicaiton_1['period'])

In [ ]:
lcm = np.lcm.reduce(task['period'])

In [ ]:
period_candidaite = list(range(min(applicaiton_1['period']), max(applicaiton_1['period'])+1, gcd))

In [ ]:
applicaton_size = list(applicaiton_1['size'] * 8)

In [ ]:
applicaton_period = list(applicaiton_1['period'])

In [ ]:
applicaton_size

In [ ]:
applicaton_period

In [ ]:
np.lcm.reduce(applicaton_period)

0.3s

In [ ]:
[400, 800, 12000]

In [ ]:
12000

In [ ]:
12000 -> 4
12800 -> 2


In [ ]:
for x in period_candidaite:
    theta / sum(applicaton_size)

In [ ]:
np.floor(period_candidaite[0] / 100000000)

In [ ]:
10000001

In [ ]:
window_candidate = []
for x in period_candidaite:
    window_size = 0
    for i, task in applicaiton_1.iterrows():
        number_task = np.ceil(x / task['period'])
        window_size += number_task * task['size'] * 8
#         if number_task > 1:
#             window_size += task['period'] * (number_task - 1)
    window_candidate.append(window_size)

In [ ]:
period_candidaite

In [ ]:
period_candidaite

## 2. Simulation

### Constant Values

In [ ]:
## Network setting
NUM_ES = 8
NUM_SW = 8
NUM_NODE = NUM_ES + NUM_SW
NUM_PORT = 4
NUM_QUEUES = 8

PROC = 1_000

## Task setting
NUM_FLOW = 8

## Global setting
GRANULARITY = 100

### initial variables

In [ ]:
net = np.zeros(shape = (NUM_NODE, NUM_NODE))
for _, row in network.iterrows():
    net[eval(row['Link'])[0], eval(row['Link'])[1]] = 1
period = list(task['period'])
size = list(task['size'])
deadline = list(task['deadline'])
MTU = np.lcm.reduce(period)

## Shortest path
def bfs_paths(graph, start, goal):
    queue = [(start, [start])]
    while queue:
        (vertex, path) = queue.pop(0)
        for _next in set(np.reshape(np.argwhere(graph[vertex] > 0),  -1)) - set(path):
            if _next == goal:
                yield path + [_next]
            else:
                queue.append((_next, path + [_next]))

In [ ]:
t_release = [0] * NUM_FLOW

### Calculate latency and jitter

Calaucate the release time of each flow.

In [ ]:
log = [[[], []] for i in range(NUM_FLOW)]

In [ ]:
# %%cython -a
# def match_time(int t, int[:, :] sche) -> int:
#     '''
#     Use binary search to quickly find the posistion of GCL
#     '''
#     gate_time = [x[0] for x in sche]
#     left = 0
#     right = len(sche) - 1
#     if t >= gate_time[right]:
#         return right

#     while True:
#         median = (left + right) // 2
#         if right - left <= 1:
#             return left
#         elif gate_time[left] <= t < gate_time[median]:
#             right = median
#         else:
#             left = median

In [ ]:
def match_time(t, sche) -> int:
    '''
    Use binary search to quickly find the posistion of GCL
    '''
    if not sche:
        return -1
    gate_time = [x[0] for x in sche]
    left = 0
    right = len(sche) - 1
    if gate_time[right] <= t < sche[-1][1]:
        return right
    elif sche[-1][1] <= t or t < gate_time[0]:
        return -1
    
    while True:
        median = (left + right) // 2
        if right - left <= 1:
            return left
        elif gate_time[left] <= t < gate_time[median]:
            right = median
        else:
            left = median

### Define Task

In [ ]:
## Shortest path
def bfs_paths(graph, start, goal):
    queue = [(start, [start])]
    while queue:
        (vertex, path) = queue.pop(0)
        for _next in set(np.reshape(np.argwhere(graph[vertex] > 0),  -1)) - set(path):
            if _next == goal:
                yield path + [_next]
            else:
                queue.append((_next, path + [_next]))

In [ ]:
for link in GCL:
    GCL[link] = sorted(GCL[link], key=lambda x:x[0], reverse=False)

In [ ]:
for link in GCL:
    temp = GCL[link]
    for i, row in enumerate(temp[:-1]):
        if row[1] >= temp[i+1][0]:
            print('overlap', link, row, temp[i+1])

In [ ]:
log = [[[], []] for i in range(NUM_FLOW)]
egress_q = {link: [[] for i in range(NUM_QUEUES)] for link, _ in GCL.items()}
mutex_lock = {link: 0 for link, _ in GCL.items()}
_pool = {link: [] for link, _ in GCL.items()}

for t in tqdm(range(0, MTU * 10, GRANULARITY)):
    if t % MTU == 0:
        print("[Net Cycle] ------ %d ----------"%(t // MTU))
        
    ## Release task
    for flow in range(NUM_FLOW):
        if t % period[flow] == 0:
            link = (route[flow][0], route[flow][1])
            egress_q[link][queue[flow][link]].append(flow)
            
    ## Timer - TODO: Replace by heap
    for link, vec in _pool.items():
        _new_vec = []
        for ct, flow in vec:
            if t >= ct:
                if link[-1] == route[flow][-1]: 
                    log[flow][1].append(t)
                    if VERBOSE:
                        print("Flow %d: Received at %d"%(flow,t))
                    continue
                elif link[0] == route[flow][0]:
                    log[flow][0].append(t)
                    if VERBOSE:
                        print("Flow %d: Released at %d"%(flow,t))
                new_link = (link[-1], _route[flow][link[-1]])
                egress_q[new_link][queue[flow][new_link]].append(flow)
            else:
                _new_vec.append((ct, flow))
        _pool[link] = _new_vec
    
    # Qbv
    for link, sche in GCL.items():
        current_t = t % MTU
        index = match_time(current_t, sche)
        if index == -1:
            continue
        s, e, q = sche[index]
        if t >= mutex_lock[link] and egress_q[link][q]:
            trans_delay = size[egress_q[link][q][0]] * 8
            if e - current_t >= trans_delay:
                out = egress_q[link][q].pop(0)
                _pool[link].append((t + trans_delay + PROC, out))
                mutex_lock[link] = t + trans_delay

### Define Schedule